<a href="https://colab.research.google.com/github/lamalex/cs722-pointnet/blob/main/cs722-pointnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!curl -L http://3dvision.princeton.edu/projects/2014/3DShapeNets/ModelNet10.zip -o ModelNet10.zip
!unzip -q ModelNet10.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  451M  100  451M    0     0  15.4M      0  0:00:29  0:00:29 --:--:-- 16.5M


In [ ]:
!python -V
!pip install pycodestyle pycodestyle_magic flake8 nptyping

Python 3.6.9
     |████████████████████████████████| 51kB 3.6MB/s 
     |████████████████████████████████| 81kB 4.2MB/s 
     |████████████████████████████████| 71kB 4.9MB/s 
     |████████████████████████████████| 51kB 4.6MB/s 


In [ ]:
%load_ext pycodestyle_magic

In [ ]:
import os
import random
import numpy as np
from enum import Enum
from pathlib import Path
from itertools import islice
# from functools import reduce
from nptyping import NDArray
from typing import Tuple, List, Dict, Optional, Callable

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

import plotly.graph_objects as go
from plotly.subplots import make_subplots

**ModelNet10** contains CAD models from 10 categories. The models are described using .off files.
.off is a simple format where:
- The first line has OFF to mark the file as .off
- The 2nd line is # vertices, # faces, # edges
- List of vertices (X, Y, Z, W)
- List of faces
- List of edges

In [ ]:
path = Path("ModelNet10")

In [ ]:
# Would have liked to use a DataClass here to define
# a simple Mesh type with vertices/faces accessors, but
# LO: Python 3.6.9 vs Python 3.7
# A Tuple of lists will work, but I don't love it.

def read_off(file: str) -> Tuple[List[int], List[float]]:
    '''
    https://en.wikipedia.org/wiki/OFF_(file_format)
    Simple geometry definition file containing lists
    of vertices, faces, and edges

    Args:
        file: File system path to be read

    Returns:
        Tuple of (vertices, faces)
    '''
    if 'OFF' != file.readline().strip():
        raise('Not a valid OFF header')

    n_verts, n_faces, _ = tuple(
        [int(s) for s in file.readline().strip().split(' ')]
    )
    vertices = [
        [float(s) for s in file.readline().strip().split(' ')]
        for _ in range(n_verts)
    ]
    faces = [
        [int(s) for s in file.readline().strip().split(' ')][1:]
        for _ in range(n_faces)
    ]

    return vertices, faces

## Generating a point cloud from 3D cad object
CAD objects are defined using vertices and faces but we can sample faces
to generate a point cloud.

Note: Data directly from a raw source like LiDAR or Kinect scanner would
be a point cloud (which is the motivation for this paper!), but CAD programs
from which our dataset is derived define their objects in a different format.

In [ ]:
def _triangle_area(a: NDArray[np.float64], b: NDArray[np.float64],
                   c: NDArray[np.float64]) -> np.float64:
    '''
    Calculate area of a triangle by its vertices
    https://en.wikipedia.org/wiki/Heron%27s_formula

    Args:
        a: First vertiex defining triangle
        b: Second vertex defining triangle
        c: Third vertex defining triangle

    Returns:
        Area of triangle defined by a, b, c
    '''
    side_a = np.linalg.norm(a - b)
    side_b = np.linalg.norm(b - c)
    side_c = np.linalg.norm(c - a)
    s = 0.5 * (side_a + side_b + side_c)
    return max(s * (s - side_a) * (s - side_b) * (s - side_c), 0)**0.5


def _sample_point(a: NDArray[np.float64],
                  b: NDArray[np.float64],
                  c: NDArray[np.float64]) -> Tuple[np.float64,
                                                   np.float64,
                                                   np.float64]:
    '''
    Sample points on surface of a triangle defined by 3 vertices

    Args:
        a: First vertex defining triangle
        b: Second vertex defining triangle
        c: Third vertex defining triangle

    Returns:
        Point defined by (x,y,z) located on surface of triangle defined
        by input (a, b, c)
    '''
    # barycentric coordinates on a triangle
    # https://mathworld.wolfram.com/BarycentricCoordinates.html
    s, t = sorted([random.random(), random.random()])
    f = lambda i: s * a[i] + (t-s) * b[i] + (1-t) * c[i]
    return (f(0), f(1), f(2))


class Pointcloudify:
    '''
    Process a mesh into a point cloud
    '''
    def __init__(self, samples_per_face: int = 1024):
        '''
        Args:
            samples_per_face: number of points per face to sample
        '''
        self.samples_per_face = samples_per_face

    def __call__(self, mesh: Tuple[List[int], List[float]]) -> NDArray:
        '''
        Sample
        '''
        verts, faces = mesh
        verts = np.array(verts)

        # calculate areas of all faces in our mesh
        areas = (_triangle_area(verts[face[0]],
                                verts[face[1]],
                                verts[face[2]]) for face in faces)

        sampled_faces = (random.choices(faces,
                                        weights=areas,
                                        k=self.samples_per_face))

        return np.array([_sample_point(verts[sampled_face[0]],
                                       verts[sampled_face[1]],
                                       verts[sampled_face[2]]) for sampled_face
                         in sampled_faces])


with open(f'{path}/chair/train/chair_0001.off', 'r') as night_stand_file:
    mesh = read_off(night_stand_file)

pointcloud = Pointcloudify()(mesh)

# Visualizing our 3D model
in 3 ways

In [ ]:
# Same plot type 3 times
plots = [[{'type': 'scene'} for _ in range(3)]]
marker_style = {'size': 1.5 }

fig = make_subplots(rows=1,
                    cols=3,
                    specs=plots,
                    subplot_titles=("3D Mesh", "Vertices Only", "Point Cloud"))

fig.add_trace(
    go.Mesh3d(
        x=[x[0] for x in mesh[0]],
        y=[y[1] for y in mesh[0]],
        z=[z[2] for z in mesh[0]],
        i=[i[0] for i in mesh[1]],
        j=[j[1] for j in mesh[1]],
        k=[k[2] for k in mesh[1]],
    ),
    row=1,
    col=1)

fig.add_trace(
    go.Scatter3d(
        x=[x[0] for x in mesh[0]],
        y=[y[1] for y in mesh[0]],
        z=[z[2] for z in mesh[0]],
        mode='markers',
        marker=marker_style
    ),
    row=1,
    col=2)

fig.add_trace(
    go.Scatter3d(
        x=[x[0] for x in pointcloud],
        y=[y[1] for y in pointcloud],
        z=[z[2] for z in pointcloud],
        mode='markers',
        marker=marker_style
    ),
    row=1,
    col=3)

fig.update_layout(height=600, width=1024, showlegend=False)
fig.show()

# PointNet

## Data loading and preprocessing

In [128]:
class DataSplitType(Enum):
    '''
    ModelNet has training and test data split into different folders
    Rather than using magic strings use an enum
    '''
    TRAIN = 'train'
    TEST = 'test'


def _list_classes_from_path(root_dir: Path) -> Dict[str, int]:
    '''
    Args:
        root_dir: Path to 3D model data with substructure of form
            root_dir/(class)/(data split)/class_xxxx.off

    Returns:
        Dictionary sorted alphabetically with key: classname
        and value class # based on alphanumeric ordering
    '''
    return {klass: i for i, klass in
            enumerate([klass for klass in sorted(os.listdir(root_dir))
                       if os.path.isdir(root_dir/klass)])}


# Python 3.8 has TypedDict which WOULD BE NICE
def _get_files_for_class(root_dir: Path,
                         class_map: Tuple[str, int],
                         split_type: 'DataSplitType') -> List[Dict[str, any]]:
    '''
    Args:
        root_dir: Path to 3D model data with substructure of form
            root_dir/(class)/(data split)/class_xxxx.off
        class_map: (Class name, id) for which t
        split_type: test/train

    Returns:
        Mapping of .off file path and object class for all files of class
        in split_type
    '''
    class_name, class_id = class_map
    return [{
        'path': path/class_name/split_type.value/fname,
        'class': class_id
      } for fname in os.listdir(root_dir/class_name/split_type.value)
      if fname.endswith('.off')]


class ToTensor:
    '''
    Torch has its own ToTensor class in transforms module
    but torch.transforms.ToTensor creates a 3D tensor instead of 2
    '''
    def __call__(self, pointcloud):
        return torch.from_numpy(pointcloud)

def base_transform_pipeline() -> transforms.Compose:
    '''
    Returns:
        Transformation pipeline
    '''
    return transforms.Compose([Pointcloudify(), ToTensor()])


class PointCloudDataSet(Dataset):
    '''
    Point cloud data loading helper
    https://pytorch.org/tutorials/beginner/data_loading_tutorial.html
    '''
    def __init__(self,
                 root_dir: str,
                 split_type: 'DataSplitType',
                 transform: Optional[Callable] = base_transform_pipeline()):
        '''
        Args:
            root_dir: Path to dataset
            split_type: Which split to load
            transform: Optional transformation to be applied to sample
        '''
        self.classes_map = _list_classes_from_path(root_dir)
        self.datafiles = [datafile for class_map in self.classes_map.items()
                          for datafile in _get_files_for_class(
                             path,
                             class_map,
                             split_type)]
        self.transform_pipeline = transform

    def _process(self, file: str) -> torch.Tensor:
        '''
        Load .off file and perform any preprocessing

        Args:
            file: path to .off file to load and process

        Returns:
            post-transformation tensor from .off file
        '''
        mesh = read_off(file)

        if self.transform_pipeline is None:
            return base_transform_pipeline()(mesh)
        return self.transform_pipeline(mesh)

    def __len__(self):
        return len(self.datafiles)

    def __getitem__(self, idx):
        with open(self.datafiles[idx]['path'], 'r') as f:
            pointcloud = self._process(f)

        return {'pointcloud': pointcloud, 'class': self.datafiles[idx]['class']}

In [130]:
training_data = PointCloudDataSet(
    root_dir=path,
    split_type=DataSplitType.TRAIN)

BATCH_SIZE = 32
training_dataloader = DataLoader(
    dataset=training_data,
    batch_size=BATCH_SIZE,
    shuffle=True)

## T-Net

In [138]:
def window(seq, n=2):
    '''
    Returns a sliding window (of width n) over data from the iterable
    s -> (s0,s1,...s[n-1]), (s1,s2,...,sn), ...
    '''
    it = iter(seq)
    result = tuple(islice(it, n))
    if len(result) == n:
        yield result
    for elem in it:
        result = result[1:] + (elem,)
        yield result


class TNet(nn.Module):
    '''
    Regression network for predicting a k x k transformation matrix.
    A sequence of batch normalized CNNs, MLPs, and a max pooling layer.
    '''
    def __init__(self, k: int):
        '''
        Args:
            k: Input layer dimensionality
        '''
        super(TNet, self).__init__()
        dims = [k, 64, 128, 1024, 512, 256, k**2]
        cnn_dims = window(dims[:4])
        fc_dims = window(dims[3:])

        self.k = k
        self.cnn = [nn.Conv1d(i, o, 1) for (i, o) in cnn_dims]
        self.fc = [nn.Linear(i, o) for (i, o) in fc_dims]
        self.bn = [nn.BatchNorm1d(d) for d in dims[1:-1]]

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        '''
        Predict (bs, k, k) affine transformation matrix
        for projecting model into normalized space.

        Args:
            x: A batch of n examples with shape (batch size, n, k)
        
        Returns:
            Tensor of (bs, k, k) affine transformation matrix
        '''
        batchsize = x.size()[0]
        bn_iter = iter(self.bn)

        # Extremely obtuse one-liner for sequential application of
        # ReLU(BatchNorm(CNN)) for each CNN layer, but I finally found a use
        # for the walrus operator so I'm keeping it.
        # EXCEPT OMFG GOOGLE COLAB IS PYTHON 3.6 SO I CAN'T USE := 🤬
        # x = reduce(lambda x,
        #             f: (cnn := f[0],
        #                 bn := f[1],
        #                 F.relu(bn(cnn(x)))
        #                )[-1],
        #             zip(self.cnn, bn_iter), x)

        # Same as above in a much more readable and Python 3.6 friendly version
        for i in range(len(self.cnn)):
            bn = bn_iter.__next__()
            x = F.relu(bn(self.cnn[i](x)))

        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, 1024)

        # IT STINGS EVERY TIME
        # x = reduce(lambda x,
        #            f: (fc := f[0],
        #                bn := f[1],
        #                F.relu(bn(fc(x)))
        #               )[-1],
        #            zip(self.fc, bn_iter), x)
        for i in range(len(self.fc[:-1])):
            bn = bn_iter.__next__()
            x = F.relu(bn(self.fc[i](x)))

        x = self.fc[-1](x)
        ident = torch.eye(self.k, requires_grad=True).repeat(batchsize, 1, 1)
        x = x.view(-1, self.k, self.k) + ident

        return x

In [139]:
tnet = TNet(k=3)
tnet.to("cpu")
tnet.train()

output = None
for data in training_dataloader:
    inputs = data['pointcloud'].to("cpu").float().transpose(1,2)
    output = tnet(inputs)
tnet.eval()
output

tensor([[[ 1.0102e+00, -3.6701e-02,  8.5977e-02],
         [-1.0947e-01,  1.4674e+00,  2.2548e-01],
         [ 6.8917e-02, -3.7084e-01,  7.9786e-01]],

        [[ 1.3342e+00,  9.7414e-02,  1.1485e-01],
         [-5.1475e-02,  1.0862e+00, -1.6456e-01],
         [-6.6344e-02,  2.4132e-01,  4.8041e-01]],

        [[ 8.7389e-01,  2.9330e-01,  1.2814e-01],
         [ 8.9045e-02,  8.3327e-01, -1.8509e-01],
         [ 2.7492e-03, -2.4754e-01,  8.0800e-01]],

        [[ 9.7563e-01,  3.1331e-01, -1.5510e-01],
         [-2.6617e-01,  2.0623e+00,  2.9936e-01],
         [-3.4870e-01,  2.3673e-01,  2.4197e-01]],

        [[ 8.0970e-01,  4.2620e-01,  1.3150e-01],
         [ 1.1664e-01,  9.2852e-01, -1.6178e-01],
         [ 6.7199e-02, -1.1388e-01,  8.8857e-01]],

        [[ 1.0886e+00,  1.0103e-01,  3.1795e-01],
         [ 2.0152e-02,  1.2157e+00,  3.0404e-02],
         [ 7.6404e-02, -4.2501e-01,  8.9178e-01]],

        [[ 9.2170e-01,  1.7521e+00,  9.4138e-01],
         [-7.4255e-02,  6.2544e-01, -1